<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Handle importing/installing, both local and on Colab** 

In [5]:
import sys
ON_COLAB = 'google.colab' in sys.modules
if ON_COLAB:
    # Do stuff that only needs to happen on colab
    !pip install pyspark  # noqa
    pass
else:
    # Do stuff that only needs to happen on local computer
    pass

Rerun the code cell below to use the latest version of the python files!

In [6]:
import importlib

def import_modules():
    if ON_COLAB:
        print("Running in Colab: importing from GitHub.")
        # Clone the repository
        !git clone https://github.com/FedorTaggenbrock/data_intensive_systems.git

        # Change the current working directory to the cloned repository directory
        %cd /content/data_intensive_systems

        # Pull the latest changes from the repository
        !git pull origin main

        # Change back to the original working directory
        %cd /content

        # Add the path of the modules to sys.path
        sys.path.insert(0, "/content/data_intensive_systems/src")
    else:
        print("Running locally, importing from local.")
        # Append parent folder to path
        sys.path.append('../src')

    # Import and reload modules iteratively
    module_names = ['clustering', 'evaluate_clustering', 'generate_data',
                    'parse_data', 'main_tests']
    for module_name in module_names:
        module = importlib.import_module(module_name)
        importlib.reload(module)
        globals()[module_name] = module

import_modules()


Running locally, importing from local.


In [8]:
main_tests.run_all_tests()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Cannot assign requested address: bind: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:438)
	at sun.nio.ch.Net.bind(Net.java:430)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:225)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:600)
	at io.netty.

Test all functions you want inside the run_all_tests() during development, for small sample sizes.


Code below is for actual result generation later, so that we can easily reuse intermediate values.

**Create a spark instance**

In [5]:
spark_settings = {}
spark_instance = make_spark()

NameError: ignored

**Load and parse data**

In [ ]:
# Load and parse data
data_for_spark = parse_data()

**Perform clustering**

In [ ]:
# Run clustering and parameter tuning
clustering_settings = {
    'clustering_algorithm':'kmodes',
}
clustering_centroid_outcomes = run_clustering(
    spark_instance=spark_instance,
    data_for_spark, clustering_settings)

In [ ]:
# Evaluate clustering results
clustering_evaluation = evaluate_clustering(clustering_centroid_outcomes)

**Display results**

In [ ]:
# Display results (maybe only best result)
clustering_result = display_results(clustering_evaluation